In [1]:
import sigpde.torch as sig
import torch
import math
from samplers import *
device = torch.device('cuda:0')

In [2]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 1)
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 1)

In [12]:
x = generate(8, 512, 50, device=device)
y = generate(4, 512, 50, device=device)
z, w = sample(8)
z = add_time(z)
w = add_time(w)
x = add_time(x)
y = add_time(y)